In [1]:
### Required Libraries ###
### This is how the requests library is imported to AWS Lambda ###
from datetime import datetime
import requests
#from dateutil.relativedelta import relativedelta
#from botocore.vendored import requests

In [2]:
### This section is for helper functions and data validation ###

def build_validation_result(is_valid, violated_slot, message_content):
    """
    Defines an internal validation message structured as a python dictionary.
    """
    if message_content is None:
        return {"isValid": is_valid, "violatedSlot": violated_slot}

    return {
        "isValid": is_valid,
        "violatedSlot": violated_slot,
        "message": {"contentType": "PlainText", "content": message_content},
    }
## Validates the data provided by the customer. 
    
def validate_data(debtTotal,humanAge,incomeMonthly,debtAvailable, debtmonthly):

    if userAge is not None:
        userAge = parse_float(
            userAge
        )  # Since parameters are strings it's important to cast values
        if userAge < 0:
            return build_validation_result(
                False,
                "userAge",
                "This service requires you to be 18 or older. Please enter a valid age.",
            )
        
      
    if numberDependents is not None:
        numberDependents = parse_float(
            numberDependents
        )  # Since parameters are strings it's important to cast values
        if numberDependents < 0:
            return build_validation_result(
                False,
                "numberDependents"
                "The total number of Dependents amount cannot be negative. "
                "Please provide a number greater than or equal to 0.",
            )
    if numberMortgages is not None:
        numberMortgages = parse_float(
            numberMortgages
        )  # Since parameters are strings it's important to cast values
        if numberMortgages < 0:
            return build_validation_result(
                False,
                "numberMortgages"
                "The total number of mortgages cannot be negative. "
                "Please provide a number greater than or equal to 0.",
            )
    if monthlyIncome is not None:
        monthlyIncome = parse_float(
            monthlyIncome
        )  # Since parameters are strings it's important to cast values
        if monthlyIncome < 0:
            return build_validation_result(
                False,
                "monthlyIncome"
                "The total monthly income cannot be negative. "
                "Please provide a number greater than or equal to 0.",
            )
    if creditUtilization is not None:
        creditUtilization = parse_float(
            creditUtilization
        )  # Since parameters are strings it's important to cast values
        if creditUtilization < 0:
            return build_validation_result(
                False,
                "creditUtilization"
                "The total of credit utilization cannot be negative. "
                "Please provide a number greater than or equal to 0.",
            )
    if debtRatio is not None:
        debtRatio = parse_float(
            debtRatio
        )  # Since parameters are strings it's important to cast values
        if debtRatio < 0:
            return build_validation_result(
                False,
                "debtRatio"
                "The total debt ratio cannot be negative. "
                "Please provide a number greater than or equal to 0.",
            )
    if thirtyLate is not None:
        thirtyLate = parse_float(
            thirtyLate
        )  # Since parameters are strings it's important to cast values
        if thirtyLate < 0:
            return build_validation_result(
                False,
                "thirtyLate"
                "The total number of times late cannot be negative. "
                "Please provide a number greater than or equal to 0.",
            )
    if ninetyLate is not None:
        ninetyLate = parse_float(
            ninetyLate
        )  # Since parameters are strings it's important to cast values
        if ninetyLate < 0:
            return build_validation_result(
                False,
                "ninetyLate"
                "The total number of times late cannot be negative. "
                "Please provide a number greater than or equal to 0.",
            )
        
    

    return build_validation_result(True, None, None)


In [13]:
### this section is for helper functions to control the conversation to response events
def get_slots(intent_request):
    """
    Fetch all the slots and their values from the current intent.
    """
    return intent_request["currentIntent"]["slots"]
## not sure what this does
def elicit_slot(session_attributes, intent_name, slots, slot_to_elicit, message):
    """
    Defines an elicit slot type response.
    """

    return {
        "sessionAttributes": session_attributes,
        "dialogAction": {
            "type": "ElicitSlot",
            "intentName": intent_name,
            "slots": slots,
            "slotToElicit": slot_to_elicit,
            "message": message,
        },
    }

## not sure what this does
def delegate(session_attributes, slots):
    """
    Defines a delegate slot type response.
    """

    return {
        "sessionAttributes": session_attributes,
        "dialogAction": {"type": "Delegate", "slots": slots},
    }


def close(session_attributes, fulfillment_state, message):
    """
    Defines a close slot type response.
    """

    response = {
        "sessionAttributes": session_attributes,
        "dialogAction": {
            "type": "Close",
            "fulfillmentState": fulfillment_state,
            "message": message,
        },
    }

    return response

def predict_delinquency(intent_request):
    """
    Performs dialog management and fulfillment for predicting financial future.
    """
    # Gets slots' values
    user_Age = get_slots(intent_request)["userAge"]
    number_Dependents = get_slots(intent_request)["numberDependents"]
    number_Mortgages = get_slots(intent_request)["numberMortgages"]
    monthly_Income = get_slots(intent_request)["monthlyIncome"]
    credit_Utilization = get_slots(intent_request)["creditUtilization"]
    debt_Ratio = get_slots(intent_request)["debtRatio"]
    thirty_Late = get_slots(intent_request)["thirtyLate"]
    ninety_Late = get_slots(intent_request)["ninetyLate"]

    # Gets the invocation source, for Lex dialogs "DialogCodeHook" is expected.
    source = intent_request["invocationSource"]

    if source == "DialogCodeHook":
        # This code performs basic validation on the supplied input slots.

        # Gets all the slots
        slots = get_slots(intent_request)

        # Validates user's input using the validate_data function
        validation_result = validate_data(userAge, numberDependents, numberMortgages,monthlyIncome,creditUtilization,debtRatio,thirtyLate,ninetyLate)

        # If the data provided by the user is not valid,
        # the elicitSlot dialog action is used to re-prompt for the first violation detected.
        if not validation_result["isValid"]:
            slots[validation_result["violatedSlot"]] = None  # Cleans invalid slot

            # Returns an elicitSlot dialog to request new data for the invalid slot
            return elicit_slot(
                intent_request["sessionAttributes"],
                intent_request["currentIntent"]["name"],
                slots,
                validation_result["violatedSlot"],
                validation_result["message"],
            )

        # Fetch current session attributes
        output_session_attributes = intent_request["sessionAttributes"]

        # Once all slots are valid, a delegate dialog is returned to Lex to choose the next course of action.
        return delegate(output_session_attributes, get_slots(intent_request))
    

    ##This is the API call to our Heroku instance to allow for the ML model to process the customer input data. 
    res = requests.post(
    url='https://givingitago.herokuapp.com/',
    json={
        'user_Age': userAge,
        'number_Dependents': numberDependents,
        'number_Mortgages': numberMortgages,
        'monthly_Income': monthlyIncome,
        'credit_Utilization': creditUtilization,
        'debt_Ratio': debtRatio,
        'thirty_Late': thirtyLate,
        'ninety_Late': ninetyLate,
            }
        )
   
    delinquency_prediction = requests.json()
    

    # Return a message with the Genie's result. Need to identify where Fulfilled is coming from/going to
    return close(
        intent_request["sessionAttributes"],
        "Fulfilled",
        {
            "contentType": "PlainText",
            "content": """Thanks for waiting;
            Our models are predicting you {} experiencing significant financial turmoil in the next 2 years. Have a good day!.
            """.format(
                delinquency_prediction
            ),
        },
    )


In [4]:


### Intents Dispatcher ###
def dispatch(intent_request):
    """
    Called when the user specifies an intent for this bot.
    """

    # Get the name of the current intent
    intent_name = intent_request["currentIntent"]["name"]

    # Dispatch to bot's intent handlers
    if intent_name == " DeliquencyGenie":
        return predict_delinquency(intent_request)

    raise Exception("Intent with name " + intent_name + " not supported")
    
### Main Handler ###
def lambda_handler(event, context):
    """
    Route the incoming request based on intent.
    The JSON body of the request is provided in the event slot.
    """

    return dispatch(event)

In [6]:
res = requests.post(
url='https://givingitago.herokuapp.com/',
json={
    'user_Age': userAge,
    'number_Dependents': numberDependents,
    'number_Mortgages': numberMortgages,
    'monthly_Income': monthlyIncome,
    'credit_Utilization': creditUtilization,
    'debt_Ratio': debtRatio,
    'thirty_Late': thirtyLate,
    'ninety_Late': ninetyLate,
            }
        )
delinquency_prediction = requests.json()

NameError: name 'userAge' is not defined